## License
This file is part of the project hyperMusic. All of hyperMusic code is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version. hyperMusic is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details. You should have received a copy of the GNU General Public License along with hyperMusic. If not, see <https://www.gnu.org/licenses/>.

In [1]:
import os
import glob
import sys 

import bct as bct
import matplotlib.pyplot as plt
from mne.stats import fdr_correction
import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns
sns.set_style("white")

### First, we get some helper and anaysis functions going on...

In [5]:
'''
This function is used to create a distribution based on permutated paired t-tests
condition = 'Insula Left,Parietal Right,leader,follower,beta,beta'
for condition in conditions:
    perm_cond = grand_grand_df[grand_grand_df['conditions'] == condition]
    perm_ste = perm_cond[['ste', 'baseline_ste']].values
    experimental_value, _ = sp.stats.ttest_rel(perm_ste[:, 0], perm_ste[:, 1]) # experimental value of paired t-test
    perm_distribution = np.array([monte_carlo_dist(perm_ste) for i in range(1000)])
    p_value = (experimental_value < perm_distribution).sum()/64.
    if p_value == 0.0:
        out_df.loc[out_df[out_df['conditions'] == condition].index[0], 'p_value'] = 1/64.
    else:
        out_df.loc[out_df[out_df['conditions'] == condition].index[0], 'p_value'] = p_value
'''

def info_source(row):
    if row['sub_source'] == row['sub_leader']: 
        return 'leader'
    else:
        return 'follower'

def info_target(row):
    if row['sub_target'] == row['sub_leader']:
        return 'leader'
    else:
        return 'follower'
    
def monte_carlo_dist_unpaired(matrix_inputs):
    # concatenate both columns into one long vector
    n_vector = matrix_inputs.shape[0]
    long_vector = matrix_inputs.reshape((n_vector*2))

    # permute values
    perm_vector = np.random.permutation(long_vector)

    # unpack arrays
    a_out = perm_vector[n_vector:]
    b_out = perm_vector[:n_vector]

    # get ind t-test
    t_out, _ = sp.stats.ttest_ind(a_out, b_out)
    return t_out

# this bit of code runs paired t-tests instead of independent tests for the MonteCarlo simulation
def monte_carlo_dist(matrix_inputs):
    """Randomly swap entries in two arrays."""
    # indices to swap
    swap_inds = np.random.random(size = matrix_inputs.shape[0]) < 0.5
    
    # unpack the matrix
    a = matrix_inputs[:, 0]
    b = matrix_inputs[:, 1]
    
    # make copies of arrays a and b for output
    a_out = np.copy(a)
    b_out = np.copy(b)
    
    # swap values
    a_out[swap_inds] = b[swap_inds]
    b_out[swap_inds] = a[swap_inds]
    
    # calculate paired t-test
    t_out, _ = sp.stats.ttest_rel(a_out, b_out)

    return t_out

# Table 1: Participants' information

In [6]:
# get participants data
participants = pd.read_csv('data/participant_music.csv')
participants = participants[participants['Pair'] != 'P01']
participants = participants[participants['Pair'] != 'P02']
participants = participants[participants['Pair'] != 'P06']
participants = participants.reset_index()
del participants['index']

pairs = participants['Pair'].tolist()
n = len(pairs)/2
sub_df = pd.DataFrame(data=np.zeros((n, 13)), 
                      columns=['pair', 'age_1', 'age_2', 'formal_1', 
                               'formal_2', 'ensemble_1', 'ensemble_2', 
                               'conservatory_1', 'conservatory_2', 
                               'profesional_1', 'profesional_2',
                               'current_ensemble_1', 'current_ensemble_2'])              
sub_df['pair'] = pairs[::2]
sub_df['age_1'] = participants['age'][::2].tolist()
sub_df['age_2'] = participants['age'][1::2].tolist()
sub_df['formal_1'] = participants['formal_training'][::2].tolist()
sub_df['formal_2'] = participants['formal_training'][1::2].tolist()
sub_df['ensemble_1'] = participants['ensemble_training'][::2].tolist()
sub_df['ensemble_2'] = participants['ensemble_training'][1::2].tolist()
sub_df['conservatory_1'] = participants['conservatory'][::2].tolist()
sub_df['conservatory_2'] = participants['conservatory'][1::2].tolist()
sub_df['current_ensemble_1'] = participants['ensemble'][::2].tolist()
sub_df['current_ensemble_2'] = participants['ensemble'][1::2].tolist()
sub_df['profesional_1'] = participants['profesional'][::2].tolist()
sub_df['profesional_2'] = participants['profesional'][1::2].tolist()
sub_df

,pair,age_1,age_2,formal_1,formal_2,ensemble_1,ensemble_2,conservatory_1,conservatory_2,profesional_1,profesional_2,current_ensemble_1,current_ensemble_2
0,P03,23,20,15,12,10,8,Y,Y,N,Y,Y,N
1,P04,29,32,10,10,7,5,Y,Y,N,N,N,N
2,P05,20,18,10,15,6,4,Y,Y,N,N,Y,Y
3,P08,21,24,15,15,4,8,Y,Y,Y,N,N,N
4,P11,38,28,15,15,20,13,Y,Y,N,Y,N,N
5,P09,21,23,14,18,4,2,Y,Y,Y,Y,Y,N


# Analysis 1: Participants descriptive statistics

In [7]:
# pairs descriptive statistics: means and std's
mean_age = sub_df[['age_1', 'age_2']].values.mean()
std_age = sub_df[['age_1', 'age_2']].values.std()
mean_play = sub_df[['formal_1', 'formal_2']].values.mean()
std_play = sub_df[['formal_1', 'formal_2']].values.std()
mean_ensemble = sub_df[['ensemble_1', 'ensemble_2']].values.mean()
std_ensemble = sub_df[['ensemble_1', 'ensemble_2']].values.std()

# pairs descriptive statistics: absolute mean differences and std's
mean_diff = abs(sub_df[['age_2']].values - sub_df[['age_1']].values).mean()
std_diff= abs(sub_df[['age_2']].values - sub_df[['age_1']].values).std()
mean_diff_play = abs(sub_df[['formal_2']].values - sub_df[['formal_1']].values).mean()
std_diff_play = abs(sub_df[['formal_2']].values - sub_df[['formal_1']].values).std()
mean_diff_ens = abs(sub_df[['ensemble_2']].values - sub_df[['ensemble_1']].values).mean()
std_diff_ens = abs(sub_df[['ensemble_2']].values - sub_df[['ensemble_1']].values).std()

# pair descriptive statistics: characteristics
conserv_conc = (sub_df['conservatory_1'] == sub_df['conservatory_2']).sum()
prof_conc = (sub_df['profesional_1'] == sub_df['profesional_2']).sum()
ensemble_conc = (sub_df['current_ensemble_1'] == sub_df['current_ensemble_2']).sum()

descriptive_stats = (mean_age, std_age, mean_diff, std_diff,
                     mean_play, std_play, mean_diff_play, std_diff_play,
                     mean_ensemble, std_ensemble, mean_diff_ens, std_diff_ens, 
                     conserv_conc, prof_conc, ensemble_conc)

print('''
We tested 6 pianist dyads. 
Mean age: %f ± %f
Absolute mean age difference: %f ± %f

Mean formal music training: %f ± %f
Absolute mean formal music training difference: %f ± %f

Mean ensemble music training: %f ± %f
Absolute mean ensemble music training difference: %f ± %f

Conservatory education: %d/6 concordant
Currently working as profesional musicians: %d/6 concordant
Currently playing in an ensemble: %d/6 concordant'''%descriptive_stats)


We tested 6 pianist dyads. 
Mean age: 24.750000 ± 5.643950
Absolute mean age difference: 3.833333 ± 2.793842

Mean formal music training: 13.666667 ± 2.460804
Absolute mean formal music training difference: 2.000000 ± 2.081666

Mean ensemble music training: 7.583333 ± 4.733891
Absolute mean ensemble music training difference: 3.166667 ± 1.863390

Conservatory education: 6/6 concordant
Currently working as profesional musicians: 3/6 concordant
Currently playing in an ensemble: 4/6 concordant


# Analysis 2: Per pair connectivity hyperbrain networks (experimental data, baseline, and scrambled pairs)

First, we take every person's csv's and dump it into grand csv files per delay

In [71]:
# We start by setting up a couple parameters
delays = ['3', '30', '150'] # Only looking at 20.81ms, 200ms, 1000ms
delays_seconds = ['20ms', '200ms', '1000ms']

# loop through delays
for i_delay in range(len(delays)):
    
    # Create grand matrix for the original data corresponding to that delay 
    file_list = glob.glob('/Users/hectorOrozco/Desktop/hM_analysis/6-STE_fixed_delayed/csv_delay_corrected/*'+delays[i_delay]+'.csv')
    grand_df = pd.read_csv(file_list[0], index_col = 0)
    for file in file_list[1:]:
        temp = pd.read_csv(file, index_col = 0)
        grand_df = grand_df.append(temp)

    # Get info direction from the csv
    grand_df['info_source'] = grand_df.apply(info_source, axis = 1)
    grand_df['info_target'] = grand_df.apply(info_target, axis = 1)

    # Output grand csv
    grand_df.to_csv('results/5.fixed_delay_trial/grand_csv_fixed_delay'+delays_seconds[i_delay]+'.csv')
    print(grand_df.shape)

    # Create grand baseline matrix for corresponding delay 
    file_list = glob.glob('/Users/hectorOrozco/Desktop/hM_analysis/7-STE_fixed_delay_baseline/baseline_grand_matrices_fixed/*'+delays[i_delay]+'.csv')
    baseline_df = pd.read_csv(file_list[0], index_col = 0)
    for file in file_list[1:]:
        temp = pd.read_csv(file, index_col = 0)
        baseline_df = baseline_df.append(temp)

    # Output grand csv
    baseline_df.to_csv('results/5.fixed_delay_trial/baseline_csv_fixed_delay'+delays_seconds[i_delay]+'.csv')
    print(baseline_df.shape)

    # Create grand matrix for scrambled data corresponding to delay 
    file_list = glob.glob('/Users/hectorOrozco/Desktop/hM_analysis/8-STE_fixed_delayed_scrambled/fixed_csv_scrambled_matrices/*'+delays[i_delay]+'.csv')
    scrambled_df = pd.read_csv(file_list[0], index_col = 0)

    for file in file_list[1:]:
        temp = pd.read_csv(file, index_col = 0)
        scrambled_df = scrambled_df.append(temp)

    # Get info direction from the csv
    scrambled_df['info_source'] = scrambled_df.apply(info_source, axis = 1)
    scrambled_df['info_target'] = scrambled_df.apply(info_target, axis = 1)

    # Output grand csv
    scrambled_df.to_csv('results/5.fixed_delay_trial/scrambled_csv_fixed_delay'+delays_seconds[i_delay]+'.csv')
    print(scrambled_df.shape)

(1699200, 15)
(86400, 9)
(1368000, 17)
(1699200, 15)
(86400, 9)
(1368000, 17)
(1699200, 15)
(86400, 9)
(1368000, 17)


We output at this point homophonic vs polyphonic pieces for baseline, scrambled pairs, and original data just to doublec check everything is Ok 

In [72]:
# Experimental data
delays = ['3', '30', '150'] # Only looking at 20.81ms, 200ms, 1000ms
delays_seconds = ['20ms', '200ms', '1000ms']
pairs = ['P03', 'P04', 'P05', 'P08', 'P09', 'P11']
sources_aranged = ['Prefrontal Left', 'Insula Left', 'Motor Left', 'Temporal Left', 'Parietal Left', 'Occipital Left',
                   'Prefrontal Right', 'Insula Right', 'Motor Right', 'Temporal Right', 'Parietal Right', 'Occipital Right']
freq_aranged = ['delta', 'theta', 'alpha', 'beta', 'gamma']

# Loop through all the delays and pairs 
for i_delay in range(len(delays)):
    grand_df = pd.read_csv('results/5.fixed_delay_trial/grand_csv_fixed_delay'+delays_seconds[i_delay]+'.csv', index_col = 0)
    for i_pair in range(len(pairs)):
        pair_grand_df = grand_df[grand_df['pair'] == pairs[i_pair]].copy()

        # Average homophonic and polyphonic pieces separately 
        pair_matrix = pair_grand_df.groupby(['neuro_source', 'neuro_target', 'info_source', 'info_target', 'freq_source', 'freq_target', 
                                             'duo_type'], as_index=False)['ste'].mean()

        # Consider these variables as categorical so they are ordered correctly when creating the figure
        pair_matrix.loc[:, 'neuro_source'] = pd.Categorical(pair_matrix['neuro_source'], 
                                                         categories=sources_aranged, ordered = True)
        pair_matrix.loc[:, 'neuro_target'] = pd.Categorical(pair_matrix['neuro_target'], 
                                                         categories=sources_aranged, ordered = True)
        pair_matrix.loc[:, 'freq_source'] = pd.Categorical(pair_matrix['freq_source'], 
                                                        categories=freq_aranged, ordered = True)
        pair_matrix.loc[:, 'freq_target'] = pd.Categorical(pair_matrix['freq_target'], 
                                                        categories=freq_aranged, ordered = True)

        homo_matrix = pair_matrix[pair_matrix['duo_type'] == 'h']
        homo_matrix = homo_matrix.pivot_table(index = ['freq_source', 'info_source', 'neuro_source'], 
                                            columns = ['freq_target', 'info_target', 'neuro_target'], 
                                            values = 'ste')

        # Create normal figure
        fig, axs = plt.subplots(1,1, figsize=(30, 30))
        sns.heatmap(homo_matrix, cmap='Blues', square=True, linewidths=.5, ax = axs, cbar_kws = {'shrink': 0.5})
        plt.hlines(range(12, 121, 12), 120, 0, linewidth = 0.5, linestyle = '--')
        plt.hlines(range(24, 121, 24), 120, 0, linewidth = 1)
        plt.vlines(range(12, 121, 12), 120, 0, linewidth = 0.5, linestyle = '--')
        plt.vlines(range(24, 121, 24), 120, 0, linewidth = 1)
        axs.set_title('Homophonic, original data %s, %s delay' % ((pairs[i_pair]), delays_seconds[i_delay]), fontsize=15)
        plt.tight_layout()
        fig.savefig('results/5.fixed_delay_trial/'+pairs[i_pair]+'_homo_original_fixed_delay'+delays_seconds[i_delay]+'.png', 
                    bbox_inches = 'tight',pad_inches = 0.1)

        # Set diagonal to zero
        np.fill_diagonal(homo_matrix.values, 0) 
        fig, axs = plt.subplots(1,1, figsize=(30, 30))
        sns.heatmap(homo_matrix, cmap='Blues', square=True, linewidths=.5, ax = axs, cbar_kws = {'shrink': 0.5})
        plt.hlines(range(12, 121, 12), 120, 0, linewidth = 0.5, linestyle = '--')
        plt.hlines(range(24, 121, 24), 120, 0, linewidth = 1)
        plt.vlines(range(12, 121, 12), 120, 0, linewidth = 0.5, linestyle = '--')
        plt.vlines(range(24, 121, 24), 120, 0, linewidth = 1)
        axs.set_title('Homophonic, original data, diag = 0 %s, %s delay' % ((pairs[i_pair]), delays_seconds[i_delay]), fontsize=15)
        fig.savefig('results/5.fixed_delay_trial/'+pairs[i_pair]+'_homo_diag0_fixed_delay'+delays_seconds[i_delay]+'.png', 
                    bbox_inches = 'tight',pad_inches = 0.1)

        # Set within connections to zero
        between_matrix = pair_matrix[pair_matrix['duo_type'] == 'h'].copy()
        between_matrix = between_matrix[between_matrix['info_source'] != between_matrix['info_target']]
        between_matrix = between_matrix.pivot_table(index = ['freq_source', 'info_source', 'neuro_source'], 
                                            columns = ['freq_target', 'info_target', 'neuro_target'], 
                                            values = 'ste')
        fig, axs = plt.subplots(1,1, figsize=(30, 30))
        sns.heatmap(between_matrix, cmap='Blues', square=True, linewidths=.5, ax = axs, cbar_kws = {'shrink': 0.5})
        plt.hlines(range(12, 121, 12), 120, 0, linewidth = 0.5, linestyle = '--')
        plt.hlines(range(24, 121, 24), 120, 0, linewidth = 1)
        plt.vlines(range(12, 121, 12), 120, 0, linewidth = 0.5, linestyle = '--')
        plt.vlines(range(24, 121, 24), 120, 0, linewidth = 1)
        axs.set_title('Homophonic, original data, between connections, %s, %s delay' % ((pairs[i_pair]), delays_seconds[i_delay]), fontsize=15)
        fig.savefig('results/5.fixed_delay_trial/'+pairs[i_pair]+'_homo_between_fixed_delay'+delays_seconds[i_delay]+'.png', 
                    bbox_inches = 'tight',pad_inches = 0.1)
        plt.close('all')
        
        poly_matrix = pair_matrix[pair_matrix['duo_type'] == 'p']
        poly_matrix = poly_matrix.pivot_table(index = ['freq_source', 'info_source', 'neuro_source'], 
                                            columns = ['freq_target', 'info_target', 'neuro_target'], 
                                            values = 'ste')

        # Create normal figure
        fig, axs = plt.subplots(1,1, figsize=(30, 30))
        sns.heatmap(poly_matrix, cmap='Blues', square=True, linewidths=.5, ax = axs, cbar_kws = {'shrink': 0.5})
        plt.hlines(range(12, 121, 12), 120, 0, linewidth = 0.5, linestyle = '--')
        plt.hlines(range(24, 121, 24), 120, 0, linewidth = 1)
        plt.vlines(range(12, 121, 12), 120, 0, linewidth = 0.5, linestyle = '--')
        plt.vlines(range(24, 121, 24), 120, 0, linewidth = 1)
        axs.set_title('Polyphonic, original data %s, %s delay' % ((pairs[i_pair]), delays_seconds[i_delay]), fontsize=15)
        plt.tight_layout()
        fig.savefig('results/5.fixed_delay_trial/'+pairs[i_pair]+'_poly_original_fixed_delay'+delays_seconds[i_delay]+'.png', 
                    bbox_inches = 'tight',pad_inches = 0.1)

        # Set diagonal to zero
        np.fill_diagonal(poly_matrix.values, 0) 
        fig, axs = plt.subplots(1,1, figsize=(30, 30))
        sns.heatmap(poly_matrix, cmap='Blues', square=True, linewidths=.5, ax = axs, cbar_kws = {'shrink': 0.5})
        plt.hlines(range(12, 121, 12), 120, 0, linewidth = 0.5, linestyle = '--')
        plt.hlines(range(24, 121, 24), 120, 0, linewidth = 1)
        plt.vlines(range(12, 121, 12), 120, 0, linewidth = 0.5, linestyle = '--')
        plt.vlines(range(24, 121, 24), 120, 0, linewidth = 1)
        axs.set_title('Polyphonic, original data, diag = 0 %s, %s delay' % ((pairs[i_pair]), delays_seconds[i_delay]), fontsize=15)
        fig.savefig('results/5.fixed_delay_trial/'+pairs[i_pair]+'_poly_diag0_fixed_delay'+delays_seconds[i_delay]+'.png', 
                    bbox_inches = 'tight',pad_inches = 0.1)

        # Set within connections to zero
        poly_matrix = pair_matrix[pair_matrix['duo_type'] == 'h'].copy()
        poly_matrix = poly_matrix[poly_matrix['info_source'] != poly_matrix['info_target']]
        poly_matrix = poly_matrix.pivot_table(index = ['freq_source', 'info_source', 'neuro_source'], 
                                            columns = ['freq_target', 'info_target', 'neuro_target'], 
                                            values = 'ste')
        fig, axs = plt.subplots(1,1, figsize=(30, 30))
        sns.heatmap(poly_matrix, cmap='Blues', square=True, linewidths=.5, ax = axs, cbar_kws = {'shrink': 0.5})
        plt.hlines(range(12, 121, 12), 120, 0, linewidth = 0.5, linestyle = '--')
        plt.hlines(range(24, 121, 24), 120, 0, linewidth = 1)
        plt.vlines(range(12, 121, 12), 120, 0, linewidth = 0.5, linestyle = '--')
        plt.vlines(range(24, 121, 24), 120, 0, linewidth = 1)
        axs.set_title('Polyphonic, original data, between connections, %s, %s delay' % ((pairs[i_pair]), delays_seconds[i_delay]), fontsize=15)
        fig.savefig('results/5.fixed_delay_trial/'+pairs[i_pair]+'_poly_between_fixed_delay'+delays_seconds[i_delay]+'.png', 
                    bbox_inches = 'tight',pad_inches = 0.1)
        plt.close('all')

In [73]:
# Baseline
delays = ['3', '30', '150'] # Only looking at 20.81ms, 200ms, 1000ms
delays_seconds = ['20ms', '200ms', '1000ms']
pairs = ['P03', 'P04', 'P05', 'P08', 'P09', 'P11']
sources_aranged = ['Prefrontal Left', 'Insula Left', 'Motor Left', 'Temporal Left', 'Parietal Left', 'Occipital Left',
                   'Prefrontal Right', 'Insula Right', 'Motor Right', 'Temporal Right', 'Parietal Right', 'Occipital Right']
freq_aranged = ['delta', 'theta', 'alpha', 'beta', 'gamma']

# Loop through all the delays and pairs 
for i_delay in range(len(delays)):
    grand_df = pd.read_csv('results/5.fixed_delay_trial/baseline_csv_fixed_delay'+delays_seconds[i_delay]+'.csv', index_col = 0)
    for i_pair in range(len(pairs)):
        pair_grand_df = grand_df[grand_df['pair'] == pairs[i_pair]].copy()
        # Consider these variables as categorical so they are ordered correctly when creating the figure
        pair_grand_df.loc[:, 'neuro_source'] = pd.Categorical(pair_grand_df['neuro_source'], 
                                                         categories=sources_aranged, ordered = True)
        pair_grand_df.loc[:, 'neuro_target'] = pd.Categorical(pair_grand_df['neuro_target'], 
                                                         categories=sources_aranged, ordered = True)
        pair_grand_df.loc[:, 'freq_source'] = pd.Categorical(pair_grand_df['freq_source'], 
                                                        categories=freq_aranged, ordered = True)
        pair_grand_df.loc[:, 'freq_target'] = pd.Categorical(pair_grand_df['freq_target'], 
                                                        categories=freq_aranged, ordered = True)

        baseline_matrix = pair_grand_df.pivot_table(index = ['freq_source', 'sub_source', 'neuro_source'], 
                                            columns = ['freq_target', 'sub_target', 'neuro_target'], 
                                            values = 'ste')

        # Create normal figure
        fig, axs = plt.subplots(1,1, figsize=(30, 30))
        sns.heatmap(baseline_matrix, cmap='Blues', square=True, linewidths=.5, ax = axs, cbar_kws = {'shrink': 0.5})
        plt.hlines(range(12, 121, 12), 120, 0, linewidth = 0.5, linestyle = '--')
        plt.hlines(range(24, 121, 24), 120, 0, linewidth = 1)
        plt.vlines(range(12, 121, 12), 120, 0, linewidth = 0.5, linestyle = '--')
        plt.vlines(range(24, 121, 24), 120, 0, linewidth = 1)
        axs.set_title('Baseline, original data %s, %s delay' % ((pairs[i_pair]), delays_seconds[i_delay]), fontsize=15)
        plt.tight_layout()
        fig.savefig('results/5.fixed_delay_trial/'+pairs[i_pair]+'_baseline_original_fixed_delay'+delays_seconds[i_delay]+'.png', 
                    bbox_inches = 'tight',pad_inches = 0.1)

        # Set diagonal to zero
        np.fill_diagonal(baseline_matrix.values, 0) 
        fig, axs = plt.subplots(1,1, figsize=(30, 30))
        sns.heatmap(baseline_matrix, cmap='Blues', square=True, linewidths=.5, ax = axs, cbar_kws = {'shrink': 0.5})
        plt.hlines(range(12, 121, 12), 120, 0, linewidth = 0.5, linestyle = '--')
        plt.hlines(range(24, 121, 24), 120, 0, linewidth = 1)
        plt.vlines(range(12, 121, 12), 120, 0, linewidth = 0.5, linestyle = '--')
        plt.vlines(range(24, 121, 24), 120, 0, linewidth = 1)
        axs.set_title('Baseline, original data, diag = 0 %s, %s delay' % ((pairs[i_pair]), delays_seconds[i_delay]), fontsize=15)
        fig.savefig('results/5.fixed_delay_trial/'+pairs[i_pair]+'_baseline_diag0_fixed_delay'+delays_seconds[i_delay]+'.png', 
                    bbox_inches = 'tight',pad_inches = 0.1)

        # Set within connections to zero
        between_matrix = pair_grand_df[pair_grand_df['sub_source'] != pair_grand_df['sub_target']]
        between_matrix = between_matrix.pivot_table(index = ['freq_source', 'sub_source', 'neuro_source'], 
                                            columns = ['freq_target', 'sub_target', 'neuro_target'], 
                                            values = 'ste')
        fig, axs = plt.subplots(1,1, figsize=(30, 30))
        sns.heatmap(between_matrix, cmap='Blues', square=True, linewidths=.5, ax = axs, cbar_kws = {'shrink': 0.5})
        plt.hlines(range(12, 121, 12), 120, 0, linewidth = 0.5, linestyle = '--')
        plt.hlines(range(24, 121, 24), 120, 0, linewidth = 1)
        plt.vlines(range(12, 121, 12), 120, 0, linewidth = 0.5, linestyle = '--')
        plt.vlines(range(24, 121, 24), 120, 0, linewidth = 1)
        axs.set_title('Baseline, original data, between connections, %s, %s delay' % ((pairs[i_pair]), delays_seconds[i_delay]), fontsize=15)
        fig.savefig('results/5.fixed_delay_trial/'+pairs[i_pair]+'_baseline_between_fixed_delay'+delays_seconds[i_delay]+'.png', 
                    bbox_inches = 'tight',pad_inches = 0.1)
        plt.close('all')

In [74]:
# Scrambled pairs
delays = ['3', '30', '150'] # Only looking at 20.81ms, 200ms, 1000ms
delays_seconds = ['20ms', '200ms', '1000ms']
pairs = ['P03', 'P04', 'P05', 'P08', 'P09', 'P11']
sources_aranged = ['Prefrontal Left', 'Insula Left', 'Motor Left', 'Temporal Left', 'Parietal Left', 'Occipital Left',
                   'Prefrontal Right', 'Insula Right', 'Motor Right', 'Temporal Right', 'Parietal Right', 'Occipital Right']
freq_aranged = ['delta', 'theta', 'alpha', 'beta', 'gamma']

# Loop through all the delays and pairs 
for i_delay in range(len(delays)):
    grand_df = pd.read_csv('results/5.fixed_delay_trial/scrambled_csv_fixed_delay'+delays_seconds[i_delay]+'.csv', index_col = 0)
    for i_pair in range(len(pairs)):
        pair_grand_df = grand_df[grand_df['pair_a'] == pairs[i_pair]].copy()

        # Average all pieces together
        scrambled_matrix = pair_grand_df.groupby(['neuro_source', 'neuro_target', 'info_source', 'info_target', 'freq_source', 'freq_target'], as_index=False)['ste'].mean()

        # Consider these variables as categorical so they are ordered correctly when creating the figure
        scrambled_matrix.loc[:, 'neuro_source'] = pd.Categorical(scrambled_matrix['neuro_source'], 
                                                         categories=sources_aranged, ordered = True)
        scrambled_matrix.loc[:, 'neuro_target'] = pd.Categorical(scrambled_matrix['neuro_target'], 
                                                         categories=sources_aranged, ordered = True)
        scrambled_matrix.loc[:, 'freq_source'] = pd.Categorical(scrambled_matrix['freq_source'], 
                                                        categories=freq_aranged, ordered = True)
        scrambled_matrix.loc[:, 'freq_target'] = pd.Categorical(scrambled_matrix['freq_target'], 
                                                        categories=freq_aranged, ordered = True)

        scrambled_matrix = scrambled_matrix.pivot_table(index = ['freq_source', 'info_source', 'neuro_source'], 
                                            columns = ['freq_target', 'info_target', 'neuro_target'], 
                                            values = 'ste')

        # Create normal figure
        fig, axs = plt.subplots(1,1, figsize=(30, 30))
        sns.heatmap(scrambled_matrix, cmap='Blues', square=True, linewidths=.5, ax = axs, cbar_kws = {'shrink': 0.5})
        plt.hlines(range(12, 121, 12), 120, 0, linewidth = 0.5, linestyle = '--')
        plt.hlines(range(24, 121, 24), 120, 0, linewidth = 1)
        plt.vlines(range(12, 121, 12), 120, 0, linewidth = 0.5, linestyle = '--')
        plt.vlines(range(24, 121, 24), 120, 0, linewidth = 1)
        axs.set_title('Averaged pieces, scrambled data %s, %s delay' % ((pairs[i_pair]), delays_seconds[i_delay]), fontsize=15)
        plt.tight_layout()
        fig.savefig('results/5.fixed_delay_trial/'+pairs[i_pair]+'_avg_scrambled_fixed_delay'+delays_seconds[i_delay]+'.png', 
                    bbox_inches = 'tight',pad_inches = 0.1)

        # Set within connections to zero
        scrambled_between_matrix = pair_grand_df[pair_grand_df['sub_source'] != pair_grand_df['sub_target']].copy()
        scrambled_between_matrix = scrambled_between_matrix.groupby(['neuro_source', 'neuro_target', 'info_source', 'info_target', 'freq_source', 'freq_target'], as_index=False)['ste'].mean()
        scrambled_between_matrix = scrambled_between_matrix.pivot_table(index = ['freq_source', 'info_source', 'neuro_source'], 
                                            columns = ['freq_target', 'info_target', 'neuro_target'], 
                                            values = 'ste')
        fig, axs = plt.subplots(1,1, figsize=(30, 30))
        sns.heatmap(scrambled_between_matrix, cmap='Blues', square=True, linewidths=.5, ax = axs, cbar_kws = {'shrink': 0.5})
        plt.hlines(range(12, 121, 12), 120, 0, linewidth = 0.5, linestyle = '--')
        plt.hlines(range(24, 121, 24), 120, 0, linewidth = 1)
        plt.vlines(range(12, 121, 12), 120, 0, linewidth = 0.5, linestyle = '--')
        plt.vlines(range(24, 121, 24), 120, 0, linewidth = 1)
        axs.set_title('Averaged pieces, scrambled data, between connections, %s, %s delay' % ((pairs[i_pair]), delays_seconds[i_delay]), fontsize=15)
        fig.savefig('results/5.fixed_delay_trial/'+pairs[i_pair]+'_avg_scrambled_between_fixed_delay'+delays_seconds[i_delay]+'.png', 
                    bbox_inches = 'tight',pad_inches = 0.1)
        plt.close('all')

/Users/hectorOrozco/anaconda/lib/python2.7/site-packages/matplotlib/pyplot.py:516: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


# Analysis 3: Are there differences between playing and baseline and actual pairs and shuffled pairs? (Average across all pieces and only look at between connections)

We use permutations to evaluate the statistical significance of the difference between baseline and playing together, and playing together vs scrambled participants

In [936]:
delays = ['3', '30', '150'] # Only looking at 20.81ms, 200ms, 1000ms
delays_seconds = ['20ms', '200ms', '1000ms']

for i_delay in range(len(delays)):
    # get grand dataframes 
    playing_df = pd.read_csv('results/5.fixed_delay_trial/grand_csv_fixed_delay'+delays_seconds[i_delay]+'.csv', index_col = 0)
    baseline_df = pd.read_csv('results/5.fixed_delay_trial/baseline_csv_fixed_delay'+delays_seconds[i_delay]+'.csv', index_col = 0)
    scrambled_df = pd.read_csv('results/5.fixed_delay_trial/scrambled_csv_fixed_delay'+delays_seconds[i_delay]+'.csv', index_col = 0)

    # get top .33% of the between connections based on the grand average 
    playing_mean_df = playing_df.groupby(['neuro_source', 'neuro_target', 'info_source', 'info_target', 'freq_source', 'freq_target', 'pair'], 
                                        as_index=False)['ste'].mean()
    playing_mean_df = playing_mean_df.groupby(['neuro_source', 'neuro_target', 'info_source', 'info_target', 'freq_source', 'freq_target'], 
                                              as_index=False)['ste'].mean()
    playing_mean_df = playing_mean_df[playing_mean_df['info_source'] != playing_mean_df['info_target']] #only take between connections
    playing_mean_df = playing_mean_df.sort_values(by=['ste'], ascending = False).reset_index()
    playing_top_df = playing_mean_df.iloc[:48, :] # Only taking the .33% top between connections per delay
    conditions = playing_top_df.iloc[:,1:7].apply(lambda x: ",".join(x.astype(str)), axis=1).tolist() #'Motor Right,Motor Right,leader,follower,beta,beta'
    playing_df = playing_df.groupby(['neuro_source', 'neuro_target', 'info_source', 'info_target', 'freq_source', 'freq_target', 'pair'], 
                                        as_index=False)['ste'].mean()
    playing_df['temp'] = playing_df.iloc[:,:6].apply(lambda x: ",".join(x.astype(str)), axis=1).tolist()
    playing_top_grand_df = playing_df[playing_df.temp.isin(conditions)].reset_index(drop = True)

    # get these connections from baseline dataframe tricky part: baseline doesn't have a clear direction, so we take the average between source-target and target-source
    baseline_df['temp'] = baseline_df.iloc[:,[4, 5, 2, 3]].apply(lambda x: ",".join(x.astype(str)), axis=1)
    baseline_df = baseline_df[baseline_df['sub_source'] != baseline_df['sub_target']]
    baseline_top_grand_df = baseline_df[baseline_df.temp.isin(playing_top_df.iloc[:,[1, 2, 5, 6]].apply(lambda x: ",".join(x.astype(str)), axis=1).tolist())].reset_index(drop=True)

    # We get the average a-b and b-a from the baseline values 
    baseline_top_grand_df = baseline_top_grand_df.groupby(['neuro_source', 'neuro_target', 'freq_source', 'freq_target',
                                                           'pair'], as_index=False)['ste'].mean()
    baseline_top_grand_df.loc[:, 'temp'] = baseline_top_grand_df.iloc[:,[0, 1, 2, 3, 4]].apply(lambda x: ",".join(x.astype(str)), axis=1).tolist()

    # We create the "grand grand" data frame with a column for the playing value, one for baseline,and one for scrambled
    playing_top_grand_df.loc[:,'temp'] = playing_top_grand_df.iloc[:,[0, 1, 4, 5, 6]].apply(lambda x: ",".join(x.astype(str)), axis=1).tolist()
    grand_grand_df = playing_top_grand_df.copy()
    for i_baseline in range(baseline_top_grand_df.shape[0]):
        for i_playing in range(grand_grand_df.shape[0]):
            if baseline_top_grand_df.loc[i_baseline, 'temp'] == grand_grand_df.loc[i_playing, 'temp']:
                grand_grand_df.loc[i_playing, 'baseline_ste'] = baseline_top_grand_df.loc[i_baseline, 'ste']

    # Now we get these connections from the scrambled file
    conditions = grand_grand_df.iloc[:,[0, 1, 2, 3, 4, 5]].apply(lambda x: ",".join(x.astype(str)), axis=1).tolist()
    grand_grand_df['conditions'] = conditions
    scrambled_df = scrambled_df.groupby(['neuro_source', 'neuro_target', 'info_source', 'info_target', 'freq_source', 'freq_target', 'pair_a'], 
                                    as_index=False)['ste'].mean()
    scrambled_df['conditions'] = scrambled_df.iloc[:,0:6].apply(lambda x: ",".join(x.astype(str)), axis=1).tolist() # "Insula Left,Parietal Right,leader,follower,beta,beta"
    scrambled_top_grand_df = scrambled_df[scrambled_df.conditions.isin(conditions)].reset_index(drop = True)
    scrambled_top_grand_df.loc[:,'temp'] = scrambled_top_grand_df.iloc[:,:7].apply(lambda x: ",".join(x.astype(str)), axis=1).tolist()
    grand_grand_df.loc[:,'temp'] = grand_grand_df.iloc[:,:7].apply(lambda x: ",".join(x.astype(str)), axis=1).tolist()
    grand_grand_df['scrambled_ste'] = 0.0
    for i_scrambled in range(scrambled_top_grand_df.shape[0]):
        for i_playing in range(grand_grand_df.shape[0]):
            if scrambled_top_grand_df.loc[i_scrambled, 'temp'] == grand_grand_df.loc[i_playing, 'temp']:
                grand_grand_df.loc[i_playing, 'scrambled_ste'] = scrambled_top_grand_df.loc[i_scrambled, 'ste']

    # We loop across the top .33% edges; first get a list of these edges
    conditions = np.unique(conditions)
    grand_grand_df = grand_grand_df.drop(['temp'], axis = 1)
    out_df = grand_grand_df.groupby(['neuro_source', 'neuro_target', 'freq_source', 'freq_target', 'info_source', 'info_target', 'conditions'], as_index=False)[('ste', 'baseline_ste', 'scrambled_ste')].mean()
    out_df['p_value_baseline'] = np.zeros(len(out_df))
    out_df['p_value_baseline_fdr'] = np.zeros(len(out_df))
    out_df['t_test_baseline'] = np.zeros(len(out_df))
    out_df['p_value_scrambled'] = np.zeros(len(out_df))
    out_df['p_value_scrambled_fdr'] = np.zeros(len(out_df))
    out_df['t_test_scrambled'] = np.zeros(len(out_df))

    # we loop across the edgest to get what we want; first baseline
    for condition in conditions:
        perm_cond = grand_grand_df[grand_grand_df['conditions'] == condition]
        perm_ste = perm_cond[['ste', 'baseline_ste']].values
        experimental_value, _ = sp.stats.ttest_rel(perm_ste[:, 0], perm_ste[:, 1]) # experimental value of ind t-test
        perm_distribution = np.array([monte_carlo_dist(perm_ste) for i in range(500)])
        perm_distribution = np.unique(perm_distribution)
        p_value = (experimental_value < perm_distribution).sum()/64.
        if p_value == 0.0:
            out_df.loc[out_df[out_df['conditions'] == condition].index[0], 'p_value_baseline'] = 1/5000.
        else:
            out_df.loc[out_df[out_df['conditions'] == condition].index[0], 'p_value_baseline'] = p_value
        out_df.loc[out_df[out_df['conditions'] == condition].index[0], 't_test_baseline'] = experimental_value  

        # scrambled
        perm_ste = perm_cond[['ste', 'scrambled_ste']].values
        experimental_value, _ = sp.stats.ttest_rel(perm_ste[:, 0], perm_ste[:, 1]) # experimental value of ind t-test
        perm_distribution = np.array([monte_carlo_dist(perm_ste) for i in range(500)])
        perm_distribution = np.unique(perm_distribution)
        p_value = (experimental_value < perm_distribution).sum()/64.
        out_df.loc[out_df[out_df['conditions'] == condition].index[0], 'p_value_scrambled'] = p_value
        out_df.loc[out_df[out_df['conditions'] == condition].index[0], 't_test_scrambled'] = experimental_value  

    # now we do a false discovery rate procedure to control for fwer
    reject_fdr, pval_fdr = fdr_correction(out_df['p_value_baseline'].values, alpha = 0.05, method = 'indep')
    out_df['p_value_baseline_fdr'] = pval_fdr  

    reject_fdr, pval_fdr = fdr_correction(out_df['p_value_scrambled'].values, alpha = 0.05, method = 'indep')
    out_df['p_value_scrambled_fdr'] = pval_fdr 

    out_df.to_csv('results/6.fixed_betweenVSbaseline_scrambled/playing_baseline_scrambled'+delays_seconds[i_delay]+'.csv')

Because none of the between connections are significant, we will not explore the correlations between them and the PMPQ. We will only plot the averaged matrices

In [339]:
delays = ['3', '30', '150'] # Only looking at 20.81ms, 200ms, 1000ms
delays_seconds = ['20ms', '200ms', '1000ms']
sources_aranged = ['Prefrontal Left', 'Insula Left', 'Motor Left', 'Temporal Left', 'Parietal Left', 'Occipital Left',
                   'Prefrontal Right', 'Insula Right', 'Motor Right', 'Temporal Right', 'Parietal Right', 'Occipital Right']
freq_aranged = ['delta', 'theta', 'alpha', 'beta', 'gamma']
pairs = ['P03', 'P04', 'P05', 'P08', 'P09', 'P11']
plt.close('all')

# Loop through all the delays and pairs 
for i_delay in range(len(delays)):

    # get grand dataframes 
    playing_df = pd.read_csv('results/5.fixed_delay_trial/grand_csv_fixed_delay'+delays_seconds[i_delay]+'.csv', index_col = 0)
    baseline_df = pd.read_csv('results/5.fixed_delay_trial/baseline_csv_fixed_delay'+delays_seconds[i_delay]+'.csv', index_col = 0)
    scrambled_df = pd.read_csv('results/5.fixed_delay_trial/scrambled_csv_fixed_delay'+delays_seconds[i_delay]+'.csv', index_col = 0)

    # Playing
    playing_matrix = playing_df.groupby(['neuro_source', 'neuro_target', 'info_source', 'info_target', 'freq_source', 'freq_target', 'pair'], 
                                as_index=False)['ste'].mean()
    playing_matrix = playing_matrix.groupby(['neuro_source', 'neuro_target', 'info_source', 'info_target', 'freq_source', 'freq_target'], 
                                as_index=False)['ste'].mean()

    # Consider these variables as categorical so they are ordered correctly when creating the figure
    playing_matrix.loc[:, 'neuro_source'] = pd.Categorical(playing_matrix['neuro_source'], 
                                                     categories=sources_aranged, ordered = True)
    playing_matrix.loc[:, 'neuro_target'] = pd.Categorical(playing_matrix['neuro_target'], 
                                                     categories=sources_aranged, ordered = True)
    playing_matrix.loc[:, 'freq_source'] = pd.Categorical(playing_matrix['freq_source'], 
                                                    categories=freq_aranged, ordered = True)
    playing_matrix.loc[:, 'freq_target'] = pd.Categorical(playing_matrix['freq_target'], 
                                                    categories=freq_aranged, ordered = True)

    playing_matrix = playing_matrix.pivot_table(index = ['freq_source', 'info_source', 'neuro_source'], 
                                        columns = ['freq_target', 'info_target', 'neuro_target'], 
                                        values = 'ste')
    vmax_full = playing_matrix.values.max()
    if i_delay == 2:
        vmax_full *= 1.3


    # Full matrix
    fig, axs = plt.subplots(1,1)
    fig.set_size_inches(7.25, 7.25)
    sns.set(font = 'sans-serif', font_scale=0.1) # you can create the labels here!
    sns.heatmap(playing_matrix, cmap='Blues', square=True, linewidths=.1, ax = axs, cbar_kws = {'shrink': 0.5}, vmax = vmax_full)
    plt.hlines(range(12, 120, 12), 120, 0, linewidth = 0.25, linestyle = '--')
    plt.hlines(range(24, 120, 24), 120, 0, linewidth = 0.5)
    plt.vlines(range(12, 120, 12), 120, 0, linewidth = 0.25, linestyle = '--')
    plt.vlines(range(24, 120, 24), 120, 0, linewidth = 0.5)
    sns.despine(fig = fig, ax = axs)
    # use matplotlib.colorbar.Colorbar object
    cbar = axs.collections[0].colorbar
    # here set the labelsize by 20
    cbar.ax.tick_params(labelsize=4, axis='both', which = 'both', direction = 'in', pad = 0.85)
    axs.spines['top'].set_visible(False)
    axs.spines['right'].set_visible(False)
    axs.get_xaxis().tick_bottom()
    axs.get_yaxis().tick_left()
    axs.xaxis.set_ticks_position('none')
    axs.yaxis.set_ticks_position('none')
    axs.tick_params(axis='both', which = 'both', direction = 'in', pad = 0.5)
    fig.savefig('results/6.fixed_betweenVSbaseline_scrambled/playing_avg_fixed_delay'+delays_seconds[i_delay]+'.eps', 
                        bbox_inches = 'tight',pad_inches = 0.1, dpi=300)

    # Between
    playing_between = playing_df.copy()
    playing_between.loc[playing_between['info_source'] == playing_between['info_target'], 'ste'] = 0.0
    playing_between = playing_between.groupby(['neuro_source', 'neuro_target', 'info_source', 'info_target', 'freq_source', 'freq_target', 'pair'], 
                                as_index=False)['ste'].mean()
    playing_between = playing_between.groupby(['neuro_source', 'neuro_target', 'info_source', 'info_target', 'freq_source', 'freq_target'], 
                                as_index=False)['ste'].mean()

    playing_between.loc[:, 'neuro_source'] = pd.Categorical(playing_between['neuro_source'], 
                                                     categories=sources_aranged, ordered = True)
    playing_between.loc[:, 'neuro_target'] = pd.Categorical(playing_between['neuro_target'], 
                                                     categories=sources_aranged, ordered = True)
    playing_between.loc[:, 'freq_source'] = pd.Categorical(playing_between['freq_source'], 
                                                    categories=freq_aranged, ordered = True)
    playing_between.loc[:, 'freq_target'] = pd.Categorical(playing_between['freq_target'], 
                                                    categories=freq_aranged, ordered = True)

    playing_between = playing_between.pivot_table(index = ['freq_source', 'info_source', 'neuro_source'], 
                                        columns = ['freq_target', 'info_target', 'neuro_target'], 
                                        values = 'ste')
    vmax_between = playing_between.values.max() * 1.2

    fig, axs = plt.subplots(1,1)
    fig.set_size_inches(7.25, 7.25)
    sns.set(font = 'sans-serif', font_scale=0.1) # you can create the labels here!
    sns.heatmap(playing_between, cmap='Blues', square=True, linewidths=.1, ax = axs, cbar_kws = {'shrink': 0.5}, vmax = vmax_between)
    plt.hlines(range(12, 120, 12), 120, 0, linewidth = 0.25, linestyle = '--')
    plt.hlines(range(24, 120, 24), 120, 0, linewidth = 0.5)
    plt.vlines(range(12, 120, 12), 120, 0, linewidth = 0.25, linestyle = '--')
    plt.vlines(range(24, 120, 24), 120, 0, linewidth = 0.5)
    sns.despine(fig = fig, ax = axs)
    # use matplotlib.colorbar.Colorbar object
    cbar = axs.collections[0].colorbar
    # here set the labelsize by 20
    cbar.ax.tick_params(labelsize=4, axis='both', which = 'both', direction = 'in', pad = 0.85)
    axs.spines['top'].set_visible(False)
    axs.spines['right'].set_visible(False)
    axs.get_xaxis().tick_bottom()
    axs.get_yaxis().tick_left()
    axs.xaxis.set_ticks_position('none')
    axs.yaxis.set_ticks_position('none')
    axs.tick_params(axis='both', which = 'both', direction = 'in', pad = 0.5)
    fig.savefig('results/6.fixed_betweenVSbaseline_scrambled/playing_avg_between_fixed_delay'+delays_seconds[i_delay]+'.eps', 
                        bbox_inches = 'tight',pad_inches = 0.1, dpi=300)
    # Baseline
    baseline_df['sub_source'] = baseline_df.iloc[:,[0, 7]].apply(lambda x: ",".join(x.astype(str)), axis=1).tolist()    
    baseline_df['sub_target'] = baseline_df.iloc[:,[1, 7]].apply(lambda x: ",".join(x.astype(str)), axis=1).tolist()

    sub_a_pairs = ['11A,P11', '3A,P03', '5A,P05', '5B,P04', '8A,P08', '9A,P09']
    sub_b_pairs = ['11B,P11', '4B,P03', '5B,P05', '6B,P04', '8B,P08', '9B,P09']
    baseline_df.loc[baseline_df['sub_source'].isin(sub_a_pairs), 'sub_source'] = 'a'
    baseline_df.loc[baseline_df['sub_target'].isin(sub_a_pairs), 'sub_target'] = 'a'
    baseline_df.loc[baseline_df['sub_source'].isin(sub_b_pairs), 'sub_source'] = 'b'
    baseline_df.loc[baseline_df['sub_target'].isin(sub_b_pairs), 'sub_target'] = 'b'

    # Average all edges
    baseline_matrix = baseline_df.groupby(['neuro_source', 'neuro_target', 'sub_source', 'sub_target', 'freq_source', 'freq_target', 'pair'], 
                                as_index=False)['ste'].mean()
    baseline_matrix = baseline_matrix.groupby(['neuro_source', 'neuro_target', 'sub_source', 'sub_target', 'freq_source', 'freq_target'], 
                                as_index=False)['ste'].mean()

    # Consider these variables as categorical so they are ordered correctly when creating the figure
    baseline_matrix.loc[:, 'neuro_source'] = pd.Categorical(baseline_matrix['neuro_source'], 
                                                     categories=sources_aranged, ordered = True)
    baseline_matrix.loc[:, 'neuro_target'] = pd.Categorical(baseline_matrix['neuro_target'], 
                                                     categories=sources_aranged, ordered = True)
    baseline_matrix.loc[:, 'freq_source'] = pd.Categorical(baseline_matrix['freq_source'], 
                                                    categories=freq_aranged, ordered = True)
    baseline_matrix.loc[:, 'freq_target'] = pd.Categorical(baseline_matrix['freq_target'], 
                                                    categories=freq_aranged, ordered = True)
    baseline_matrix = baseline_matrix.pivot_table(index = ['freq_source', 'sub_source', 'neuro_source'], 
                                        columns = ['freq_target', 'sub_target', 'neuro_target'], 
                                        values = 'ste')

    # Full matrix
    fig, axs = plt.subplots(1,1)
    fig.set_size_inches(7.25, 7.25)
    sns.set(font = 'sans-serif', font_scale=0.1) # you can create the labels here!
    sns.heatmap(baseline_matrix, cmap='Blues', square=True, linewidths=.1, ax = axs, cbar_kws = {'shrink': 0.5}, vmax = vmax_full)
    plt.hlines(range(12, 120, 12), 120, 0, linewidth = 0.25, linestyle = '--')
    plt.hlines(range(24, 120, 24), 120, 0, linewidth = 0.5)
    plt.vlines(range(12, 120, 12), 120, 0, linewidth = 0.25, linestyle = '--')
    plt.vlines(range(24, 120, 24), 120, 0, linewidth = 0.5)
    sns.despine(fig = fig, ax = axs)
    # use matplotlib.colorbar.Colorbar object
    cbar = axs.collections[0].colorbar
    # here set the labelsize by 20
    cbar.ax.tick_params(labelsize=4, axis='both', which = 'both', direction = 'in', pad = 0.85)
    axs.spines['top'].set_visible(False)
    axs.spines['right'].set_visible(False)
    axs.get_xaxis().tick_bottom()
    axs.get_yaxis().tick_left()
    axs.xaxis.set_ticks_position('none')
    axs.yaxis.set_ticks_position('none')
    axs.tick_params(axis='both', which = 'both', direction = 'in', pad = 0.5)
    fig.savefig('results/6.fixed_betweenVSbaseline_scrambled/baseline_avg_fixed_delay'+delays_seconds[i_delay]+'.eps', 
                        bbox_inches = 'tight',pad_inches = 0.1, dpi=300)

    # Between
    baseline_between = baseline_df.copy()
    baseline_between.loc[baseline_between['sub_source'] == baseline_between['sub_target'], 'ste'] = 0.0
    baseline_between = baseline_between.groupby(['neuro_source', 'neuro_target', 'sub_source', 'sub_target', 'freq_source', 'freq_target', 'pair'], 
                                as_index=False)['ste'].mean()
    baseline_between = baseline_between.groupby(['neuro_source', 'neuro_target', 'sub_source', 'sub_target', 'freq_source', 'freq_target'], 
                                as_index=False)['ste'].mean()



    # Consider these variables as categorical so they are ordered correctly when creating the figure
    baseline_between.loc[:, 'neuro_source'] = pd.Categorical(baseline_between['neuro_source'], 
                                                     categories=sources_aranged, ordered = True)
    baseline_between.loc[:, 'neuro_target'] = pd.Categorical(baseline_between['neuro_target'], 
                                                     categories=sources_aranged, ordered = True)
    baseline_between.loc[:, 'freq_source'] = pd.Categorical(baseline_between['freq_source'], 
                                                    categories=freq_aranged, ordered = True)
    baseline_between.loc[:, 'freq_target'] = pd.Categorical(baseline_between['freq_target'], 
                                                    categories=freq_aranged, ordered = True)
    baseline_between = baseline_between.pivot_table(index = ['freq_source', 'sub_source', 'neuro_source'], 
                                        columns = ['freq_target', 'sub_target', 'neuro_target'], 
                                        values = 'ste')

    fig, axs = plt.subplots(1,1)
    fig.set_size_inches(7.25, 7.25)
    sns.set(font = 'sans-serif', font_scale=0.1) # you can create the labels here!
    sns.heatmap(baseline_between, cmap='Blues', square=True, linewidths=.1, ax = axs, cbar_kws = {'shrink': 0.5}, vmax = vmax_between)
    plt.hlines(range(12, 120, 12), 120, 0, linewidth = 0.25, linestyle = '--')
    plt.hlines(range(24, 120, 24), 120, 0, linewidth = 0.5)
    plt.vlines(range(12, 120, 12), 120, 0, linewidth = 0.25, linestyle = '--')
    plt.vlines(range(24, 120, 24), 120, 0, linewidth = 0.5)
    sns.despine(fig = fig, ax = axs)
    # use matplotlib.colorbar.Colorbar object
    cbar = axs.collections[0].colorbar
    # here set the labelsize by 20
    cbar.ax.tick_params(labelsize=4, axis='both', which = 'both', direction = 'in', pad = 0.85)
    axs.spines['top'].set_visible(False)
    axs.spines['right'].set_visible(False)
    axs.get_xaxis().tick_bottom()
    axs.get_yaxis().tick_left()
    axs.xaxis.set_ticks_position('none')
    axs.yaxis.set_ticks_position('none')
    axs.tick_params(axis='both', which = 'both', direction = 'in', pad = 0.5)
    fig.savefig('results/6.fixed_betweenVSbaseline_scrambled/baseline_avg_between_fixed_delay'+delays_seconds[i_delay]+'.eps', 
                        bbox_inches = 'tight',pad_inches = 0.1, dpi=300)

    # scrambled
    # full
    scrambled_matrix = scrambled_df.groupby(['neuro_source', 'neuro_target', 'info_source', 'info_target', 'freq_source', 'freq_target', 'pair_a'], 
                                as_index=False)['ste'].mean()
    scrambled_matrix = scrambled_matrix.groupby(['neuro_source', 'neuro_target', 'info_source', 'info_target', 'freq_source', 'freq_target'], 
                                as_index=False)['ste'].mean()

    # Consider these variables as categorical so they are ordered correctly when creating the figure
    scrambled_matrix.loc[:, 'neuro_source'] = pd.Categorical(scrambled_matrix['neuro_source'], 
                                                     categories=sources_aranged, ordered = True)
    scrambled_matrix.loc[:, 'neuro_target'] = pd.Categorical(scrambled_matrix['neuro_target'], 
                                                     categories=sources_aranged, ordered = True)
    scrambled_matrix.loc[:, 'freq_source'] = pd.Categorical(scrambled_matrix['freq_source'], 
                                                    categories=freq_aranged, ordered = True)
    scrambled_matrix.loc[:, 'freq_target'] = pd.Categorical(scrambled_matrix['freq_target'], 
                                                    categories=freq_aranged, ordered = True)

    scrambled_matrix = scrambled_matrix.pivot_table(index = ['freq_source', 'info_source', 'neuro_source'], 
                                        columns = ['freq_target', 'info_target', 'neuro_target'], 
                                        values = 'ste')


    # Full matrix
    fig, axs = plt.subplots(1,1)
    fig.set_size_inches(7.25, 7.25)
    sns.set(font = 'sans-serif', font_scale=0.1) # you can create the labels here!
    sns.heatmap(scrambled_matrix, cmap='Blues', square=True, linewidths=.1, ax = axs, cbar_kws = {'shrink': 0.5}, vmax = vmax_full)
    plt.hlines(range(12, 120, 12), 120, 0, linewidth = 0.25, linestyle = '--')
    plt.hlines(range(24, 120, 24), 120, 0, linewidth = 0.5)
    plt.vlines(range(12, 120, 12), 120, 0, linewidth = 0.25, linestyle = '--')
    plt.vlines(range(24, 120, 24), 120, 0, linewidth = 0.5)
    sns.despine(fig = fig, ax = axs)
    # use matplotlib.colorbar.Colorbar object
    cbar = axs.collections[0].colorbar
    # here set the labelsize by 20
    cbar.ax.tick_params(labelsize=4, axis='both', which = 'both', direction = 'in', pad = 0.85)
    axs.spines['top'].set_visible(False)
    axs.spines['right'].set_visible(False)
    axs.get_xaxis().tick_bottom()
    axs.get_yaxis().tick_left()
    axs.xaxis.set_ticks_position('none')
    axs.yaxis.set_ticks_position('none')
    axs.tick_params(axis='both', which = 'both', direction = 'in', pad = 0.5)
    fig.savefig('results/6.fixed_betweenVSbaseline_scrambled/scrambled_avg_fixed_delay'+delays_seconds[i_delay]+'.eps', 
                        bbox_inches = 'tight',pad_inches = 0.1, dpi=300)
    
    # Between
    scrambled_between = scrambled_df.copy()
    scrambled_between.loc[scrambled_between['info_source'] == scrambled_between['info_target'], 'ste'] = 0.0
    scrambled_between = scrambled_between.groupby(['neuro_source', 'neuro_target', 'info_source', 'info_target', 'freq_source', 'freq_target', 'pair_a'], 
                                as_index=False)['ste'].mean()
    scrambled_between = scrambled_between.groupby(['neuro_source', 'neuro_target', 'info_source', 'info_target', 'freq_source', 'freq_target'], 
                                as_index=False)['ste'].mean()

    scrambled_between.loc[:, 'neuro_source'] = pd.Categorical(scrambled_between['neuro_source'], 
                                                     categories=sources_aranged, ordered = True)
    scrambled_between.loc[:, 'neuro_target'] = pd.Categorical(scrambled_between['neuro_target'], 
                                                     categories=sources_aranged, ordered = True)
    scrambled_between.loc[:, 'freq_source'] = pd.Categorical(scrambled_between['freq_source'], 
                                                    categories=freq_aranged, ordered = True)
    scrambled_between.loc[:, 'freq_target'] = pd.Categorical(scrambled_between['freq_target'], 
                                                    categories=freq_aranged, ordered = True)

    scrambled_between = scrambled_between.pivot_table(index = ['freq_source', 'info_source', 'neuro_source'], 
                                        columns = ['freq_target', 'info_target', 'neuro_target'], 
                                        values = 'ste')

    fig, axs = plt.subplots(1,1)
    fig.set_size_inches(7.25, 7.25)
    sns.set(font = 'sans-serif', font_scale=0.1) # you can create the labels here!
    
    sns.heatmap(scrambled_between, cmap='Blues', square=True, linewidths=.1, ax = axs, cbar_kws = {'shrink': 0.5}, vmax = vmax_between)
    plt.hlines(range(12, 120, 12), 120, 0, linewidth = 0.25, linestyle = '--')
    plt.hlines(range(24, 120, 24), 120, 0, linewidth = 0.5)
    plt.vlines(range(12, 120, 12), 120, 0, linewidth = 0.25, linestyle = '--')
    plt.vlines(range(24, 120, 24), 120, 0, linewidth = 0.5)
    sns.despine(fig = fig, ax = axs)
    # use matplotlib.colorbar.Colorbar object
    cbar = axs.collections[0].colorbar
    # here set the labelsize by 20
    cbar.ax.tick_params(labelsize=4, axis='both', which = 'both', direction = 'in', pad = 0.85)
    axs.spines['top'].set_visible(False)
    axs.spines['right'].set_visible(False)
    axs.get_xaxis().tick_bottom()
    axs.get_yaxis().tick_left()
    axs.xaxis.set_ticks_position('none')
    axs.yaxis.set_ticks_position('none')
    axs.tick_params(axis='both', which = 'both', direction = 'in', pad = 0.5)
    fig.savefig('results/6.fixed_betweenVSbaseline_scrambled/scrambled_avg_between_fixed_delay'+delays_seconds[i_delay]+'.eps', 
                        bbox_inches = 'tight',pad_inches = 0.1, dpi=300)


    plt.close('all')

# Analysis 4: Are there differences in terms of graph theory between the different pieces?

We average pieces and compare the trial based graph theory statistics between homophonic and polyphonic pieces

In [362]:
delays = ['3', '30', '150'] # Only looking at 20.81ms, 200ms, 1000ms
delays_seconds = ['20ms', '200ms', '1000ms']
pairs = ['P03', 'P04', 'P05', 'P08', 'P09', 'P11']
pieces = ['mel', 'can', 'clo', 'val']
duo_type = ['h', 'p']

# Loop through all the delays
for i_delay in range(len(delays)):
    playing_df = pd.read_csv('results/5.fixed_delay_trial/grand_csv_fixed_delay'+delays_seconds[i_delay]+'.csv', index_col = 0)
    grand_df = playing_df.groupby(['neuro_source', 'neuro_target', 'info_source', 'info_target', 'freq_source', 'freq_target', 'duo_type', 
                                         'piece', 'pair', 'trial'], as_index=False)['ste'].mean()  
    # Load the pmpq file -> This will help us and save us from doing weird shit down the line
    graph_df_original = pd.read_csv('data/pmpq/pmpq.csv')
    graph_df = graph_df_original[graph_df_original['pair'] != 'P01'].copy()
    graph_df = graph_df[graph_df['pair'] != 'P02']
    graph_df = graph_df[graph_df['pair'] != 'P06']
    graph_df = graph_df[graph_df['scale'] != 'anxiety']
    graph_df = graph_df.groupby(['pair', 'trial', 'piece', 'scale'], as_index=False)['score'].mean()
    graph_df['duo_type'] = 'trash_kween'
    graph_df.loc[:, 'duo_type'] = [graph_df.loc[:, 'piece'][i][0] for i in range(len(graph_df))]
    graph_df.loc[:, 'piece'] = [graph_df.loc[:, 'piece'][i][1:] for i in range(len(graph_df))]

    for i_pair in range(len(pairs)):
        for i_piece in range(len(pieces)):
            for i_trial in range(5):
                if pairs[i_pair] == 'P05' and i_trial == 3 and pieces[i_piece] == 'can':
                    continue
                elif pairs[i_pair] == 'P05' and i_trial == 4 and pieces[i_piece] == 'can':
                    continue

                grand_df_piece = grand_df[grand_df['piece'] == pieces[i_piece]]
                grand_df_piece = grand_df_piece[grand_df_piece['pair'] == pairs[i_pair]]
                current_trial = grand_df_piece[grand_df_piece['trial'] == (i_trial+1)].copy()
                current_trial = current_trial.pivot_table(index = ['freq_source', 'info_source', 'neuro_source'], 
                                                            columns = ['freq_target', 'info_target', 'neuro_target'], 
                                                            values = 'ste')
                current_trial = current_trial.values
                if pieces[i_piece] == 'mel' or pieces[i_piece] == 'val':
                    duo_type = 'h'
                elif pieces[i_piece] == 'clo' or pieces[i_piece] == 'can':
                    duo_type = 'p'
                else:
                    duo_type = 'whut'

                # First, we get the connection length matrix -- we need this one for some calculations downstream
                current_trial_length = bct.weight_conversion(current_trial, 'lengths', copy=True) # this will get us the connection length matrix needed for some of the above functions! 

                # Clustering
                avg_clustering_coef = bct.clustering_coef_wd(current_trial).mean() #  The weighted clustering coefficient is the average “intensity” of triangles around a node -> We take the average! 
                graph_df = graph_df.append({'pair': pairs[i_pair], 'trial': i_trial+1, 'piece': pieces[i_piece], 'scale': 'avg_clustering_coeff', 'score': avg_clustering_coef, 'duo_type': duo_type}, ignore_index=True)

                transitivity = bct.transitivity_wd(current_trial) # Transitivity is the ratio of ‘triangles to triplets’ in the network. (A classical version of the clustering coefficient)
                graph_df = graph_df.append({'pair': pairs[i_pair], 'trial': i_trial+1, 'piece': pieces[i_piece], 'scale': 'transitivity', 'score': transitivity, 'duo_type': duo_type}, ignore_index=True)

                # Core
                assortativity = bct.assortativity_wei(current_trial) # The assortativity coefficient is a correlation coefficient between the strengths (weighted degrees) of all nodes on two opposite ends of a link. A positive assortativity coefficient indicates that nodes tend to link to other nodes with the same or similar strength.
                graph_df = graph_df.append({'pair': pairs[i_pair], 'trial': i_trial+1, 'piece': pieces[i_piece], 'scale': 'assortativity', 'score': assortativity, 'duo_type': duo_type}, ignore_index=True)

                # Degree
                avg_strengths = bct.strengths_dir(current_trial).mean() # Node strength is the sum of weights of links connected to the node. The instrength is the sum of inward link weights and the outstrength is the sum of outward link weights.
                graph_df = graph_df.append({'pair': pairs[i_pair], 'trial': i_trial+1, 'piece': pieces[i_piece], 'scale': 'avg_node_strengths', 'score': avg_strengths, 'duo_type': duo_type}, ignore_index=True)

                # Distance
                # We first get the distance matrix to then get the characteristic path length
                current_trial_distance, _ = bct.distance_wei(current_trial_length) #The distance matrix contains lengths of shortest paths between all pairs of nodes. An entry (u,v) represents the length of shortest path from node u to node v. The average shortest path length is the characteristic path length of the network.
                char_path_length, efficiency, _, _, _ = bct.charpath(current_trial_distance, include_diagonal=False) # The characteristic path length is the average shortest path length in the network. The global efficiency is the average inverse shortest path length in the network.
                graph_df = graph_df.append({'pair': pairs[i_pair], 'trial': i_trial+1, 'piece': pieces[i_piece], 'scale': 'char_path_length', 'score': char_path_length, 'duo_type': duo_type}, ignore_index=True)
                graph_df = graph_df.append({'pair': pairs[i_pair], 'trial': i_trial+1, 'piece': pieces[i_piece], 'scale': 'efficiency', 'score': efficiency, 'duo_type': duo_type}, ignore_index=True)

                # now that we have all the statistics we want, we output the data frame just in case!
                graph_df.to_csv('results/7.graph_theory_fixed_delay/fixed_graph_theory_pertrial'+delays_seconds[i_delay]+'.csv')

Now, we test for differences between polyphonic and homophonic duos at each delay using permutations

In [938]:
delays = ['3', '30', '150'] # Only looking at 20.81ms, 200ms, 1000ms
delays_seconds = ['20ms', '200ms', '1000ms']
graph_stats = ['avg_clustering_coeff', 'avg_node_strengths', 'char_path_length', 'efficiency']

for i_delay in range(len(delays)):
    graph_df = pd.read_csv('results/7.graph_theory_fixed_delay/fixed_graph_theory_pertrial'+delays_seconds[i_delay]+'.csv', index_col= 0)

    # create output dataframe with all data
    out_data = {'graph_stat': graph_stats, 'score_h': np.zeros(len(graph_stats)), 
               'score_p': np.zeros(len(graph_stats)), 'p_value': np.zeros(len(graph_stats)), 
               'p_value_fdr': np.zeros(len(graph_stats)), 't_test': np.zeros(len(graph_stats))}
    out_df = pd.DataFrame(data = out_data)

    for graph_stat in graph_stats:
        perm_cond = graph_df[graph_df['scale'] == graph_stat].copy()
        perm_cond = perm_cond.groupby(['pair', 'duo_type'], as_index=False)['score'].mean()
        perm_h = perm_cond.loc[perm_cond['duo_type'] == 'h', 'score'].values[:, None]
        perm_p = perm_cond.loc[perm_cond['duo_type'] == 'p', 'score'].values[:, None]            
        perm_graph = np.concatenate((perm_p, perm_h), axis = 1)
        experimental_value, _ = sp.stats.ttest_rel(perm_graph[:, 0], perm_graph[:, 1]) # experimental value of ind t-test
        perm_distribution = np.array([monte_carlo_dist(perm_graph) for i in range(500)])
        perm_distribution = np.unique(perm_distribution)
        p_value = (np.abs(experimental_value) < np.abs(perm_distribution)).sum()/64. # this is a two-sided test    
        out_df.loc[out_df[out_df['graph_stat'] == graph_stat].index[0], 'p_value'] = p_value
        out_df.loc[out_df[out_df['graph_stat'] == graph_stat].index[0], 't_test'] = experimental_value
        out_df.loc[out_df[out_df['graph_stat'] == graph_stat].index[0], 'score_h'] = perm_h.mean()
        out_df.loc[out_df[out_df['graph_stat'] == graph_stat].index[0], 'score_p'] = perm_p.mean()

    # fdr to correct for multiple comparisons
    reject_fdr, pval_fdr = fdr_correction(out_df['p_value'].values, alpha = 0.05, method = 'indep')
    out_df['p_value_fdr'] = pval_fdr     
    out_df.to_csv('results/7.graph_theory_fixed_delay/graph_theory_stats'+delays_seconds[i_delay]+'.csv')

In [942]:
plt.close('all')
# Now we plot this graph theory statistics on their own 
titles = ['a', 'b', 'c', 'd']
for i_delay in range(len(delays_seconds)):
    graph_df = pd.read_csv('results/7.graph_theory_fixed_delay/fixed_graph_theory_pertrial'+delays_seconds[i_delay]+'.csv', index_col= 0)
    graph_statistics = ['avg_clustering_coeff', 'avg_node_strengths', 'char_path_length', 'efficiency']
    graph_title = ['Average Clustering Coefficient', 'Average Node Strength', 'Characteristic Path Length', 'Efficiency']
    graph_df = graph_df[graph_df.scale.isin(graph_statistics)].reset_index()

    colors_palette = sns.cubehelix_palette(3, start=1, rot=.7)
    fig, axs = plt.subplots(4,1)
    fig.set_size_inches(3.54, 7.25)
    fig.subplots_adjust(hspace = 1.03, wspace = 0.4)
    axs = axs.ravel()

    for i_graph in range(len(graph_statistics)):
        sns.violinplot(x="duo_type", y="score", data=graph_df[graph_df['scale'] == graph_statistics[i_graph]], 
                       palette = colors_palette, ax = axs[i_graph])
        axs[i_graph].spines['top'].set_visible(False)
        axs[i_graph].spines['right'].set_visible(False)
        axs[i_graph].get_xaxis().tick_bottom()
        axs[i_graph].get_yaxis().tick_left()
        #axs[i_graph].xaxis.set_ticks_position('none')
        #axs[i_graph].yaxis.set_ticks_position('none')
        axs[i_graph].tick_params(axis='both', which = 'both', length = 0.2, width = 0.2)
        axs[i_graph].set_title('%s'%titles[i_graph], loc = 'left', fontname = 'sans-serif', fontsize = 15)
        axs[i_graph].set_ylabel(graph_title[i_graph])
        axs[i_graph].set_xticklabels(["Homophonic", "Polyphonic"])
        axs[i_graph].xaxis.labelpad = 0.4
        axs[i_graph].yaxis.labelpad = 0.8
        h_where = graph_df.loc[graph_df['scale'] == graph_statistics[i_graph], 'score'].values.max()*1.3
        axs[i_graph].text(.5, h_where*1.001, '*', fontsize=12)
        axs[i_graph].hlines(h_where, 0, 1, linewidth =1)
        sns.set(font = 'sans-serif', font_scale = 0.5, style = 'white') # you can create the labels here!
    fig.savefig('results/7.graph_theory_fixed_delay/graph_stats_fixed_delay'+delays_seconds[i_delay]+'.eps', 
                                bbox_inches = 'tight',pad_inches = 0.1, dpi=300)

The values are significant, so we plot them and calculate the correlations

In [943]:
# We plot the correlations between this statistics now 
graph_statistics = ['avg_clustering_coeff', 'avg_node_strengths', 'char_path_length', 'efficiency']
pmpq_scales = ['quality', 'synchrony', 'synergy']
graph_title = ['Average Clustering Coefficient', 'Average Node Strength', 'Characteristic Path Length', 'Efficiency']
pmpq_title = ['Quality', 'Synchrony', 'Synergy']
colors_palette = sns.cubehelix_palette(2, start=1, rot=.7)

for i_delay in range(len(delays_seconds)):
    # wrangle the pmpq csv to make plotter easier
    graph_df = pd.read_csv('results/7.graph_theory_fixed_delay/fixed_graph_theory_pertrial'+delays_seconds[i_delay]+'.csv', index_col = 0)
    graph_df = graph_df[~graph_df.scale.isin(['anxiety'])]
    graph_df = graph_df[~graph_df.scale.isin(['transitivity', 'assortativity'])].reset_index(drop = True)
    graph_df.loc[:, 'conditions'] = graph_df.iloc[:,[0, 1, 2]].apply(lambda x: ",".join(x.astype(str)), axis=1).tolist()
    graph_df = graph_df[~graph_df['conditions'].isin(['P05,4,can', 'P05,5,can'])].reset_index(drop = True)
    graph_df_correlations = graph_df.loc[:2, :].copy()
    graph_df_correlations['graph_stats'] = 'place_holder'
    graph_df_correlations['stat'] = 0.0

    for i_scale in range(len(pmpq_scales)):
        for i_stat in range(len(graph_statistics)):
            temp_pmpq = graph_df[graph_df['scale'] == pmpq_scales[i_scale]].sort_values(by = ['pair', 'piece', 'trial']).reset_index(drop = True).copy()
            temp_stat = graph_df[graph_df['scale'] == graph_statistics[i_stat]].sort_values(by = ['pair', 'piece', 'trial']).reset_index(drop = True).copy()
            temp_pmpq.loc[:, 'graph_stats'] = temp_stat['scale'].tolist()
            temp_pmpq.loc[:, 'stat'] = temp_stat['score'].tolist()

            graph_df_correlations = graph_df_correlations.append(temp_pmpq)
            graph_df_correlations = graph_df_correlations.reset_index(drop = True)
            if i_scale == 0 and i_stat == 0:
                graph_df_correlations = graph_df_correlations.drop(index = [0, 1, 2]).reset_index(drop = True)
    graph_df_correlations.loc[:, 'conditions'] = graph_df_correlations.iloc[:,[3, 7]].apply(lambda x: ",".join(x.astype(str)), axis=1).tolist()

    plt.close('all')

    graph_conditions = np.unique(graph_df_correlations.loc[:, 'conditions'].tolist())

    out_data = {'duo_type': ['place_holder'], 'pmpq_scale': ['place_holder'], 'graph_stats': ['place_holder'], 'corr': 0.0, 'p_value': 0.0}
    out_df = pd.DataFrame(out_data)

    fig, axs = plt.subplots(3,4)
    fig.set_size_inches(7.25, 5)
    fig.subplots_adjust(hspace = .5, wspace = 0.5)
    axs = axs.ravel()

    i_graph = 0
    for i_scales in range(len(pmpq_scales)):
        for i_stats in range(len(graph_statistics)):
            # Get the beatiful plot
            graph_temp = graph_df_correlations[graph_df_correlations.conditions.isin([graph_conditions[i_graph]])].reset_index(drop = False).copy()
            sns.scatterplot(x="score", y="stat", hue = 'duo_type', data=graph_temp, palette = colors_palette, ax = axs[i_graph], s= 12)
            axs[i_graph].spines['top'].set_visible(False)
            axs[i_graph].spines['right'].set_visible(False)
            axs[i_graph].get_xaxis().tick_bottom()
            axs[i_graph].get_yaxis().tick_left()
            axs[i_graph].tick_params(axis='both', which = 'both', length = 0.2, width = 0.2, pad = .005)
            axs[i_graph].set_ylabel(graph_title[i_stats])
            axs[i_graph].set_xlabel(pmpq_title[i_scales])
            sns.set(font = 'sans-serif', font_scale = 0.5, style = 'white') # you can create the labels here!
            axs[i_graph].xaxis.labelpad = 0.3
            axs[i_graph].yaxis.labelpad = 0.3
            axs[i_graph].legend(loc='upper right', frameon=False, scatterpoints=1, fontsize=3, markerscale = 0.1, markerfirst = True)
            if graph_temp['graph_stats'].tolist()[0] == 'avg_clustering_coeff' or graph_temp['graph_stats'].tolist()[0] == 'efficiency':
                axs[i_graph].set_ylim(bottom = 0.0, top = 0.006) 
            i_graph += 1


            # Get the sweet taste of defeat because we all know these correlations are not there
            poly_pmpq = graph_temp[graph_temp['duo_type'] == 'p'].reset_index(drop = True)['score'].values
            poly_graph = graph_temp[graph_temp['duo_type'] == 'p'].reset_index(drop = True)['stat'].values
            poly_corr, poly_p = sp.stats.pearsonr(poly_pmpq, poly_graph)
            out_df = out_df.append({'corr': poly_corr, 'duo_type': 'p', 'graph_stats': graph_temp.graph_stats.tolist()[0], 'p_value': poly_p, 'pmpq_scale': graph_temp.scale.tolist()[0]}, ignore_index = True)

            homo_pmpq = graph_temp[graph_temp['duo_type'] == 'h'].reset_index(drop = True)['score'].values
            homo_graph = graph_temp[graph_temp['duo_type'] == 'h'].reset_index(drop = True)['stat'].values
            homo_corr, homo_p = sp.stats.pearsonr(homo_pmpq, homo_graph)
            out_df = out_df.append({'corr': homo_corr, 'duo_type': 'h', 'graph_stats': graph_temp.graph_stats.tolist()[0], 'p_value': homo_p, 'pmpq_scale': graph_temp.scale.tolist()[0]}, ignore_index = True)

    fig.savefig('results/7.graph_theory_fixed_delay/graph_stats_pmpq_corrs_fixed_delay'+delays_seconds[i_delay]+'.eps', 
                    bbox_inches = 'tight',pad_inches = 0.1, dpi=300)
    out_df = out_df.drop(index = 0).reset_index(drop = True)
    _, out_df['p_value_fdr'] = fdr_correction(out_df['p_value'].values, alpha = 0.05, method = 'indep') 
    out_df.to_csv('results/7.graph_theory_fixed_delay/correlations_pmpq_graph_fixed'+delays_seconds[i_delay]+'.csv')

In [956]:
# Surprisingly, some of these correlations ARE significant, so, I'm going to plot them on their own!
graph_statistics = ['avg_clustering_coeff', 'avg_node_strengths', 'char_path_length', 'efficiency']
pmpq_scales = ['quality']
graph_title = ['Average Clustering Coefficient', 'Average Node Strength', 'Characteristic Path Length', 'Efficiency']
pmpq_title = ['Quality']
colors_palette = sns.cubehelix_palette(4, start=1, rot=.7)

for i_delay in range(len(delays_seconds)):
    # wrangle the pmpq csv to make plotter easier
    graph_df = pd.read_csv('results/7.graph_theory_fixed_delay/fixed_graph_theory_pertrial'+delays_seconds[i_delay]+'.csv', index_col = 0)
    graph_df = graph_df[~graph_df.scale.isin(['anxiety'])]
    graph_df = graph_df[~graph_df.scale.isin(['transitivity', 'assortativity'])].reset_index(drop = True)
    graph_df.loc[:, 'conditions'] = graph_df.iloc[:,[0, 1, 2]].apply(lambda x: ",".join(x.astype(str)), axis=1).tolist()
    graph_df = graph_df[~graph_df['conditions'].isin(['P05,4,can', 'P05,5,can'])].reset_index(drop = True)
    graph_df_correlations = graph_df.loc[:2, :].copy()
    graph_df_correlations['graph_stats'] = 'place_holder'
    graph_df_correlations['stat'] = 0.0
    
    # only take the significant ones!
    for i_scale in range(len(pmpq_scales)):
        for i_stat in range(len(graph_statistics)):
            temp_pmpq = graph_df[graph_df['scale'] == pmpq_scales[i_scale]].sort_values(by = ['pair', 'piece', 'trial']).reset_index(drop = True).copy()
            temp_stat = graph_df[graph_df['scale'] == graph_statistics[i_stat]].sort_values(by = ['pair', 'piece', 'trial']).reset_index(drop = True).copy()
            temp_pmpq.loc[:, 'graph_stats'] = temp_stat['scale'].tolist()
            temp_pmpq.loc[:, 'stat'] = temp_stat['score'].tolist()

            graph_df_correlations = graph_df_correlations.append(temp_pmpq)
            graph_df_correlations = graph_df_correlations.reset_index(drop = True)
            if i_scale == 0 and i_stat == 0:
                graph_df_correlations = graph_df_correlations.drop(index = [0, 1, 2]).reset_index(drop = True)
    graph_df_correlations.loc[:, 'conditions'] = graph_df_correlations.iloc[:,[3, 7]].apply(lambda x: ",".join(x.astype(str)), axis=1).tolist()
    graph_df_correlations = graph_df_correlations[graph_df_correlations['duo_type']== 'p']

    plt.close('all')

    graph_conditions = np.unique(graph_df_correlations.loc[:, 'conditions'].tolist())

    

    fig, axs = plt.subplots(4,1)
    fig.set_size_inches(3.54, 7)
    fig.subplots_adjust(hspace = .5, wspace = 0.5)
    axs = axs.ravel()

    i_graph = 0
    for i_scales in range(len(pmpq_scales)):
        for i_stats in range(len(graph_statistics)):
            # Get the beatiful plot
            graph_temp = graph_df_correlations[graph_df_correlations.conditions.isin([graph_conditions[i_graph]])].reset_index(drop = False).copy()
            sns.regplot(x="score", y="stat", data=graph_temp, color = colors_palette[i_graph], ci = None, fit_reg = True, ax = axs[i_graph], scatter_kws = {'s':8}, line_kws = {'linewidth':1.5})
            axs[i_graph].spines['top'].set_visible(False)
            axs[i_graph].spines['right'].set_visible(False)
            axs[i_graph].get_xaxis().tick_bottom()
            axs[i_graph].get_yaxis().tick_left()
            axs[i_graph].tick_params(axis='both', which = 'both', length = 0.2, width = 0.2, pad = .005)
            axs[i_graph].set_ylabel(graph_title[i_stats])
            axs[i_graph].set_xlabel(pmpq_title[i_scales])
            sns.set(font = 'sans-serif', font_scale = 0.5, style = 'white') # you can create the labels here!
            if graph_temp['graph_stats'].tolist()[0] == 'avg_clustering_coeff' or graph_temp['graph_stats'].tolist()[0] == 'efficiency':
                axs[i_graph].set_ylim(bottom = 0.0, top = 0.005) 
            i_graph += 1


    fig.savefig('results/7.graph_theory_fixed_delay/sign_graph_stats_pmpq_corrs_fixed_delay'+delays_seconds[i_delay]+'.eps', 
                    bbox_inches = 'tight',pad_inches = 0.1, dpi=300)

# Analysis 4: Do the networks exhibit small world properties as a function of time?

We first look at the MAQ results, both pre and post (how much people liked each other before and after)

In [966]:
# First test for the difference in the maq
# Just to see what happens, we get the slops of the change between first trial and last trial and correlate that with how much they liked each other

# we first read the maq file and wrangle it
maq_df = pd.read_csv('data/questionnaires/5.MAQ.csv')
maq_df = maq_df[maq_df['pair'] != 'P01']
maq_df = maq_df[maq_df['pair'] != 'P02']
maq_df = maq_df[maq_df['pair'] != 'P06']
maq_df = maq_df.reset_index(drop = True)
maq_df = maq_df.drop(columns=['Timestamp', 'Email address', '1. I enjoyed playing music with my music partner',
                             '2. I would like to play again with my music partner', 
                             '3. When I was the "follower", I had no trouble musically accompanying my music partner'])

maq_df['score'] = maq_df.iloc[:, 3:6].values.mean(axis = 1)
maq_df = maq_df.drop(columns = ['4. I would like to become friends with my music partner', '5. If my music partner needed help, I would help them', 
                      '6. I would trust my music partner with a secret'])
maq_df = maq_df.groupby(by = ['time', 'pair']).mean()
maq_df = maq_df.reset_index()

# We check differences between pre and post maq
pre_maq = maq_df.loc[maq_df.time == 'pre', 'score'].values
post_maq = maq_df.loc[maq_df.time == 'post', 'score'].values
pre_post_maq = np.concatenate((np.expand_dims(pre_maq, axis = 1), np.expand_dims(post_maq, axis = 1)), axis = 1)
t_exp_maq, _ = sp.stats.ttest_rel(pre_post_maq[:, 1], pre_post_maq[:, 0]) # experimental value of ind t-test
perm_distribution = np.array([monte_carlo_dist(pre_post_maq) for i in range(5000)])
perm_distribution = np.unique(perm_distribution)
p_value_maq = (t_exp_maq < perm_distribution).sum()/64.
f=open("results/8.fixed_small_world/pre_post_maq_ttest.txt","w+")
out = 'The t-test value pre and post maq is %f and the pvalue is %f'%(t_exp_maq, p_value_maq)
f.write(out)
f.close()

# plot it gurl 
colors_palette = sns.cubehelix_palette(3, start=1, rot=.7)
fig, axs = plt.subplots(1,1)
fig.set_size_inches(3.54, 3.54) 
sns.set(font = 'sans-serif', font_scale = 1, style = 'white') # you can create the labels here!
sns.pointplot(x="time", y="score", data=maq_df, order = ['pre', 'post'], color = colors_palette[0], ax = axs)
axs.spines['top'].set_visible(False)
axs.spines['right'].set_visible(False)
axs.get_xaxis().tick_bottom()
axs.get_yaxis().tick_left()
axs.tick_params(axis='both', which = 'both', length = 0.2, width = 0.2, pad = .005)
axs.set_ylabel('Music Affiliation Questionnaire')
axs.set_xlabel('Time')
axs.text(.5, 5.15*1.001, '*', fontsize=12)
axs.hlines(5.15, 0, 1, linewidth =1)
fig.savefig('results/8.fixed_small_world/maq_pre_post.eps', bbox_inches = 'tight',pad_inches = 0.1, dpi=300)

In [1018]:
delays_seconds = ['20ms', '200ms', '1000ms']
titles = ['a', 'b', 'c', 'd', 'e', 'f']
plt.close('all')
delays_seconds = ['20ms', '200ms', '1000ms']
d = {'delay': ['20ms', '200ms', '1000ms']*2, 'corr': np.zeros(6), 'p_value': np.zeros(6), 'p_value_fdr': np.zeros(6), 'scale': 'place_holder'}
out_df = pd.DataFrame(data=d)
i_need_vacation = [1, 2, 3]

# Now we create the small world evolution graphs at each delay 
plt.close('all')
plt.clf()
colors_palette = sns.cubehelix_palette(6, start=1, rot=.7)
fig, axs = plt.subplots(3,2)
fig.set_size_inches(7.25, 7)
fig.subplots_adjust(hspace = 0.4, wspace = 0.4)
axs = axs.ravel()

for i_delay in range(len(delays_seconds)):
    # We first get the piece order by creating an agregate of pair + piece
    play_order = pd.read_csv('data/play_order.csv')
    play_order = play_order[play_order['pair'] != 'P01']
    play_order = play_order[play_order['pair'] != 'P02']
    play_order = play_order[play_order['pair'] != 'P06']
    play_order = play_order.reset_index(drop = True)
    play_order.loc[:, 'piece'] = [play_order.loc[:, 'piece'][i][1:] for i in range(len(play_order))]
    piece_1 = play_order[play_order['order'] == 1]
    piece_2 = play_order[play_order['order'] == 2]
    piece_3 = play_order[play_order['order'] == 3]
    piece_4 = play_order[play_order['order'] == 4]
    piece_1 = piece_1.iloc[:,[0, 1]].apply(lambda x: ",".join(x.astype(str)), axis=1).tolist()
    piece_2 = piece_2.iloc[:,[0, 1]].apply(lambda x: ",".join(x.astype(str)), axis=1).tolist()
    piece_3 = piece_3.iloc[:,[0, 1]].apply(lambda x: ",".join(x.astype(str)), axis=1).tolist()
    piece_4 = piece_4.iloc[:,[0, 1]].apply(lambda x: ",".join(x.astype(str)), axis=1).tolist()
    pieces_order = [piece_1, piece_2, piece_3, piece_4]

    # now we wrangle the graph theory stats to get both clustering coefficient and characteristic path length
    graph_statistics = ['avg_clustering_coeff', 'char_path_length']
    graph_df = pd.read_csv('results/7.graph_theory_fixed_delay/fixed_graph_theory_pertrial'+delays_seconds[i_delay]+'.csv', index_col = 0)
    graph_df = graph_df[graph_df.scale.isin(graph_statistics)].reset_index(drop = True)
    graph_df.loc[:, 'conditions'] = graph_df.iloc[:,[0, 2]].apply(lambda x: ",".join(x.astype(str)), axis=1).tolist()
    graph_df['order'] = 0
    for i_time in range(len(pieces_order)):
        graph_df.loc[graph_df['conditions'].isin(pieces_order[i_time]), 'order'] = (i_time+1)

    # get the small world coefficient and test correlation against time    
    cc = graph_df.loc[graph_df['scale'] == 'avg_clustering_coeff', 'score'].values
    cp = graph_df.loc[graph_df['scale'] == 'char_path_length', 'score'].values
    smoll_world = graph_df[graph_df['scale'] == 'avg_clustering_coeff'].copy().reset_index(drop = True)
    smoll_world['scale'] = 'small_world'
    smoll_world['score'] = cc/cp
    
    smoll_cor, smoll_p = sp.stats.pearsonr(smoll_world.order.values, smoll_world.score.values)
    out_df.loc[i_delay, 'scale'] = 'small_world'
    out_df.loc[i_delay, 'corr'] = smoll_cor
    out_df.loc[i_delay, 'p_value'] = smoll_p
    
    # plot it!
    sns.set(font = 'sans-serif', font_scale = 0.7, style = 'white') # you can create the labels here!
    sns.regplot(x="order", y="score", data=smoll_world, color = colors_palette[2*i_delay], ax = axs[2*i_delay], ci = None, fit_reg = True, scatter_kws = {'s':8}, line_kws = {'linewidth':1.5})
    axs[2*i_delay].spines['top'].set_visible(False)
    axs[2*i_delay].spines['right'].set_visible(False)
    axs[2*i_delay].get_xaxis().tick_bottom()
    axs[2*i_delay].get_yaxis().tick_left()
    axs[2*i_delay].tick_params(axis='both', which = 'both', length = 0.2, width = 0.2, pad = .005)
    axs[2*i_delay].set_ylabel('Small World Coefficient')
    axs[2*i_delay].set_ylim(bottom = 0.0, top = 0.000019) 
    axs[2*i_delay].set_xticks(np.arange(1, 5))
    axs[2*i_delay].set_yticks(np.arange(0.000005, 0.000016, 0.000005))
    axs[2*i_delay].set_xlabel('Experimental Block')    
    axs[2*i_delay].set_title('%s'%titles[i_delay], loc = 'left', fontname = 'sans-serif', fontsize = 15)
    
    # now we get the changes of how  much they liked each other vs the small world correlation
    slope_maq = pre_post_maq[:, 1] - pre_post_maq[:, 0]
    first_trial = smoll_world[smoll_world.order == 1].groupby(['pair'])['score'].mean().values
    last_trial = smoll_world[smoll_world.order == 4].groupby(['pair'])['score'].mean().values
    first_last_trial = np.concatenate((np.expand_dims(first_trial, axis = 1), np.expand_dims(last_trial, axis = 1)), axis = 1)
    slope_smoll = (first_last_trial[:, 1] - first_last_trial[:, 0]) / 3
    d = {'slope_maq': slope_maq, 'slope_smoll': slope_smoll}
    slopes_df = pd.DataFrame(data = d)
    sns.regplot(x='slope_maq', y='slope_smoll', data = slopes_df, color = colors_palette[i_delay+i_need_vacation[i_delay]], ci = None, fit_reg = True, ax = axs[i_delay+i_need_vacation[i_delay]], scatter_kws = {'s':8}, line_kws = {'linewidth':1.5})
    axs[i_delay+i_need_vacation[i_delay]].spines['top'].set_visible(False)
    axs[i_delay+i_need_vacation[i_delay]].spines['right'].set_visible(False)
    axs[i_delay+i_need_vacation[i_delay]].get_xaxis().tick_bottom()
    axs[i_delay+i_need_vacation[i_delay]].get_yaxis().tick_left()
    axs[i_delay+i_need_vacation[i_delay]].tick_params(axis='both', which = 'both', length = 0.2, width = 0.2, pad = .005)
    axs[i_delay+i_need_vacation[i_delay]].set_ylabel('Small World Slope')
    axs[i_delay+i_need_vacation[i_delay]].set_xlabel('MAQ Slope')
    axs[i_delay+i_need_vacation[i_delay]].set_xticks(np.arange(0,2.1, .5))
    axs[i_delay+i_need_vacation[i_delay]].set_title('%s'%titles[i_delay+3], loc = 'left', fontname = 'sans-serif', fontsize = 15)
    axs[i_delay+i_need_vacation[i_delay]].set_ylim(bottom = slopes_df.slope_smoll.min()*2, top = 0.000002) 
    sns.set(font = 'sans-serif', font_scale = 0.5, style = 'white') # you can create the labels here!
    
    smoll_maq_cor, smoll_maq_p = sp.stats.pearsonr(slope_maq, slope_smoll)
    out_df.loc[i_delay+3, 'scale'] = 'smoll_maq_corr'
    out_df.loc[i_delay+3, 'corr'] = smoll_maq_cor
    out_df.loc[i_delay+3, 'p_value'] = smoll_maq_p




_, out_df['p_value_fdr'] = fdr_correction(out_df['p_value'].values, alpha = 0.05, method = 'indep') 
out_df.to_csv('results/8.fixed_small_world/corr_time_smoll_world.csv')
fig.savefig('results/8.fixed_small_world/small_world_vs_time.eps', bbox_inches = 'tight',pad_inches = 0.1, dpi=300)